In [1]:
import numpy as np
from TimeTagger import *
import matplotlib.pyplot as plt
from IPython.display import clear_output
import serial
import time
import datetime

In [2]:
# Establish the serial connection
ser = serial.Serial('COM8', 9600)  # Replace 'COM3' with your Arduino's port
tagger = createTimeTagger()

print(ser.readline())
print(ser.readline())
print(ser.readline())

b'Demonstration of AccelStepper Library\r\n'
b"Send 'C' for printing the commands.\r\n"
b'Default speed: 400 steps/s, default acceleration: 800 steps/s^2.\r\n'


In [3]:
SPAD_CH = 1 # SPAD Channel
TRIG_CH = 3 # Trigger Channel

# introduce delay in trig channel
delay_hardware = 1.066e6 # in picosecond
tagger.setInputDelay(3,delay_hardware)

bin_width = 10e-12 # in seconds, check once
c = 3*1e10 #cm

nbins = int(5e-9/bin_width) #5 nanosecond of time data
rep_rate = 10 #1kHz
laser_power = np.round(40 * rep_rate,3) #mW

avg_noise_count = 2500
# important params: t_acq, bin_width, laser_power

delay = 0.1 #delay between two acquisitions in sec
stepper_speed = 1 #cm/sec


#to move 2cm, stepper motor will need 2*6400 steps. at this speed, it will move for 2*64 = 128 seconds lmao
# works ig
xdirection = 1 # 0 for left, 1 for right
ydirection = 0 # 0 for up, 1 for down
#t_acq = npulse/trigfreq #historgram acquisition time
t_acq = 0.1

In [4]:
print(f'Depth resolution: {np.round(bin_width*c,2)} cm')
print(f'Horizontal resolution: {stepper_speed*delay} cm')

# print(f'SPAD_CH: {SPAD_CH}')
# print(f'TRIG_CH: {TRIG_CH}')
# print(f'bin_width: {bin_width*1e12} picoseconds')
# print(f'nbins: {nbins}')
# #print(f'npulse: {npulse}')
# #print(f'trigfreq: {trigfreq} Hz')
# print(f'stepper_speed: {stepper_speed} steps/second')
# print(f'dist_in_cm: {total_distance} cm')
# print(f'number of acq: {n_acq} steps')
print(f't_acq: {t_acq} seconds')

Depth resolution: 0.3 cm
Horizontal resolution: 0.1 cm
t_acq: 0.1 seconds


In [5]:
#Directions are given from the perspective of entering the optical table room in FSLab
def send_command(command):
    ser.write(command.encode())

def right_step(dist,speed):
    send_command(f'R{dist} {speed}')
    print(ser.readline())

def left_step(dist,speed):
    send_command(f'E{dist} {speed}')
    print(ser.readline())

# def x_step(dist, speed):
#     dist*=6400
#     speed*=6400
#     if xdirection.lower() == "up":
#         left_step(dist,speed)
#     elif xdirection.lower() == "down":
#         right_step(dist,speed)
#     else:
#         exit()    

# def y_step(dist, speed):
#     dist*=6400
#     speed*=6400
#     if ydirection.lower() == "left":
#         left_step(dist,speed)
#     elif ydirection.lower() == "right":
#         right_step(dist,speed)
#     else:
#         exit()


def x_step(dist, speed):
    dist*=6400
    speed*=6400
    if xdirection == 0:
        left_step(dist,speed)
    elif xdirection == 1:
        right_step(dist,speed)
    else:
        exit()    

def y_step(dist, speed):
    dist*=6400
    speed*=6400
    if ydirection == 0:
        up_step(dist,speed)
    elif ydirection == 1:
        down_step(dist,speed)
    else:
        exit()

def home_set():
    send_command('X')
    print(ser.readline())


def down_step(dist,speed):
    send_command(f'D{dist} {speed}')
    print(ser.readline())


def up_step(dist,speed):
    send_command(f'U{dist} {speed}')
    print(ser.readline())


def home_step():
    send_command('H')  
    print(ser.readline())
    print(ser.readline())
    print(ser.readline())

def getlocation():
    send_command('L')
    t =  int(ser.readline().split()[-1]), int(ser.readline().split()[-1])
    #print(t)
    return t[0]/6400,t[1]/6400

def hist(t, bin_width, nbins):
    
    global SPAD_CH, TRIG_CH, tagger
    # print('a0) ',time.time())
    bin_width*=1e12
    t*=1e12 #every time data converted to ps
    hist = Histogram(tagger, SPAD_CH, TRIG_CH, bin_width, nbins)
    hist.startFor(t) #60e12 #in ps
    # print('a) ',time.time())
    while hist.isRunning():
        continue
    # print('b) ',time.time())    
    arr = np.array(hist.getData())
#    tt.getOverflowsAndClear()   
    #clear_output(wait=True)     
    # print('c) ',time.time())    
#    plt.plot(range(len(arr)),arr)
#    plt.show()
    # print('d) ',time.time())    
    # print('e) ',time.time())
    return arr # returns x in picosecond

In [6]:
# 2d code
x_dist = 14 #cm
y_res = 1 #cm
y_dist = 14 #cm
n_acq = int((x_dist/stepper_speed)/delay) # exact number of acquisitions

num_ysteps = int(y_dist / y_res)

home_set()


start_time = time.time()

time_delay_2d = []
for j in range(num_ysteps):
    x_step(x_dist,stepper_speed)
    time_array = np.arange(delay,delay*n_acq + delay,delay)

    time_delay_1d = []
    for i in range(n_acq):
        time_delay_1d.append(hist(t_acq,bin_width,nbins))
        if i%100==0:
            print(i)
        while time.time()<=start_time + time_array[i]:
            continue
    time_delay_2d.append(time_delay_1d)
    while (abs(getlocation()[0]) < x_dist and xdirection) or (getlocation()[0]>0 and not xdirection):
        continue

    y_step(y_res,stepper_speed)

    while (abs(getlocation()[1]) < y_res*(j+1)):
        continue

    xdirection = not(xdirection)
    


home_step()

b'Current positions are updated to 0.\r\n'


b'Negative direction for X-axis.\r\n'
0
100
b'Negative direction for Y-axis.\r\n'
b'Positive direction for X-axis.\r\n'
0
100
b'Negative direction for Y-axis.\r\n'
b'Negative direction for X-axis.\r\n'
0
100
b'Negative direction for Y-axis.\r\n'
b'Positive direction for X-axis.\r\n'
0
100
b'Negative direction for Y-axis.\r\n'
b'Negative direction for X-axis.\r\n'
0
100
b'Negative direction for Y-axis.\r\n'
b'Positive direction for X-axis.\r\n'
0
100
b'Negative direction for Y-axis.\r\n'
b'Negative direction for X-axis.\r\n'
0
100
b'Negative direction for Y-axis.\r\n'
b'Positive direction for X-axis.\r\n'
0
100
b'Negative direction for Y-axis.\r\n'
b'Negative direction for X-axis.\r\n'
0
100
b'Negative direction for Y-axis.\r\n'
b'Positive direction for X-axis.\r\n'
0
100
b'Negative direction for Y-axis.\r\n'
b'Negative direction for X-axis.\r\n'
0
100
b'Negative direction for Y-axis.\r\n'
b'Positive direction for X-axis.\r\n'
0
100
b'Negative direction for Y-axis.\r\n'
b'Negative direc

In [7]:
# np.savetxt(f'Z:/YRF/Experiments/Imaging/4_april_2D/{delay}secdelay_{bin_width}psbw_{t_acq}secacq.txt',time_delay_2d)

In [8]:
for j in range(np.shape(time_delay_2d)[0]):
    np.savetxt(f'Z:/YRF/Experiments/Imaging/4_april_2D/10kHz_0.1secdelay_10.0psbw_0.1secacq/{stepper_speed}cms-1_{x_dist}cm_{y_dist}cm.txt',time_delay_2d[j])

In [9]:
import scipy.io 
folder_name = 'Z:/YRF/Experiments/Imaging/4_april_2D/10kHz_0.1secdelay_10.0psbw_0.1secacq'
scipy.io.savemat(f'Z:/YRF/Experiments/Imaging/4_april_2D/10kHz_0.1secdelay_10.0psbw_0.1secacq/{stepper_speed}cms-1_{x_dist}cm_{y_dist}cm.mat', {'mydata': time_delay_2d})